In [1]:
words_file_path = '../data/tesstrain/kbd/configs/kbd.wordlist'
template_groups_file_path = '../dags/src/stemmer/data/template_groups_by_len.json'

In [2]:
with open(words_file_path, 'r') as f:
    words = f.read().split('\n')

In [3]:
from tqdm import tqdm

from dags.src.stemmer.rules.base import create_template_group_rule
from dags.src.stemmer.stemmer import Stemmer

group_rules = create_template_group_rule(template_groups_file_path)
stemmer = Stemmer(
    rules=[rule for g, rule in sorted(group_rules.items(), key=lambda x: x[0], reverse=True)]
)

from collections import defaultdict

# word2possible_stems = defaultdict(list)
word2templates = defaultdict(list)
for word in tqdm(words):
    # possible_stems = stemmer.find_possible_stems(word)
    # word2possible_stems[word].extend(possible_stems)

    templates = stemmer.find_suitable_templates(word)
    word2templates[word].extend(templates)

  6%|▋         | 27731/430057 [00:14<03:27, 1940.96it/s]


KeyboardInterrupt: 

In [14]:
# это не нужно, лучше сделать дерево через marisa_trie.RecordTrie и считать possible_stems шаблонам на лету
# import ahocorasick
# word2template_dawg = ahocorasick.Automaton()
# 
# word2possible_stems_dawg = ahocorasick.Automaton()
# for word, possible_stems in tqdm(word2possible_stems.items()):
#     word2possible_stems_dawg.add_word(word, possible_stems)
#     
# word2possible_stems_dawg.make_automaton()

100%|██████████| 430057/430057 [00:00<00:00, 902461.19it/s] 


In [15]:
# здесь ahocorasick тоже не нужен
# import ahocorasick
# word2templates_dawg = ahocorasick.Automaton()
# for word, templates in tqdm(word2templates.items()):
#     word2templates_dawg.add_word(word, templates)
# 
# word2templates_dawg.make_automaton()

100%|██████████| 430057/430057 [00:00<00:00, 1026930.93it/s]


In [51]:
# через marisa_trie.RecordTrie удобнее, но нужно сделать маппинг индексов в шаблоны рядом с деревом
# import marisa_trie
# 
# word_trie = marisa_trie.Trie(word2templates.keys())
# print(word_trie.keys('тхузэфIэкIын'))

['тхузэфIэкIын', 'тхузэфIэкIыну', 'тхузэфIэкIынур', 'тхузэфIэкIынуращи', 'тхузэфIэкIынури', 'тхузэфIэкIынут', 'тхузэфIэкIынущ', 'тхузэфIэкIынукъым', 'тхузэфIэкIынукъыми', 'тхузэфIэкIынум', 'тхузэфIэкIынтэкъым']


In [8]:
import marisa_trie

word_index_map = {}
index_template_map = {}
for i, (word, templates) in enumerate(tqdm(word2templates.items())):
    word_index_map[word] = (i,)
    index_template_map[i] = templates

word2template_trie = marisa_trie.RecordTrie('I', word_index_map.items())

100%|██████████| 430057/430057 [00:00<00:00, 1619312.82it/s]


In [9]:
q = 'тхузэфIэкIынукъыми'
index = word2template_trie.get(q)
print(index)
print(index_template_map[index[0][0]])

[(428006,)]
['*къыми', 'тхузэ*', 'тху*', 'т*ми', '***', '*ми', 'т*и']


In [10]:
word2template_trie.items('тхузэфIэкIын')

[('тхузэфIэкIынуращи', (338393,)),
 ('тхузэфIэкIынури', (113819,)),
 ('тхузэфIэкIынур', (115911,)),
 ('тхузэфIэкIынут', (117579,)),
 ('тхузэфIэкIынущ', (48344,)),
 ('тхузэфIэкIынукъыми', (428006,)),
 ('тхузэфIэкIынукъым', (63458,)),
 ('тхузэфIэкIынум', (52770,)),
 ('тхузэфIэкIыну', (22257,)),
 ('тхузэфIэкIынтэкъым', (163609,)),
 ('тхузэфIэкIын', (121245,))]

In [1]:
words_file_path = '../data/tesstrain/kbd/configs/kbd.wordlist'
template_groups_file_path = '../dags/src/stemmer/data/template_groups_by_len.json'

from dags.src.stemmer.store_builder import build_trie_stemmer_data

data = build_trie_stemmer_data(
    words_file_path=words_file_path,
    template_groups_file_path=template_groups_file_path,
    output_file_path='../dags/src/stemmer/data/store_data.pkl'
)


  3%|▎         | 14671/430057 [00:07<03:34, 1938.43it/s]


KeyboardInterrupt: 

In [1]:
from dags.src.stemmer.store import StemmerStore

store = StemmerStore()
store.load_data_from_file('../dags/src/stemmer/data/store_data.pkl')

100%|██████████| 430057/430057 [00:02<00:00, 146852.07it/s]


In [2]:
store.get_templates('тхузэфIэкIынут')

Source path:... /Users/panagoa/airflow/dags/src/stemmer/store.py
Starting var:.. self = <dags.src.stemmer.store.StemmerStore object at 0x1037e89d0>
Starting var:.. word = 'тхузэфIэкIынут'
19:11:32.146015 call        51     def _get_word_id(self, word):
19:11:32.146955 line        52         word_id = self.data.word2word_id_trie.get(word, None)
New var:....... word_id = [(117580,)]
19:11:32.147073 line        53         return word_id[0][0] if word_id else None
19:11:32.147141 return      53         return word_id[0][0] if word_id else None
Return value:.. 117580
Elapsed time: 00:00:00.001198
Source path:... /Users/panagoa/airflow/dags/src/stemmer/store.py
Starting var:.. self = <dags.src.stemmer.store.StemmerStore object at 0x1037e89d0>
Starting var:.. word_id = 117580
19:11:32.147210 call        61     def _get_template_ids(self, word_id):
19:11:32.147234 line        62         return self.data.word_id2template_ids.get(word_id, [])
19:11:32.147244 return      62         return self.da

['тхузэ*', '*ынут', '*нут', 'тху*', '***', '*ут', 'т*т']

In [3]:
store.data.word2word_id_trie.get('тхузэфIэкIынут')

[(117580,)]

In [9]:
store.data.word2word_id_trie.items('тхузэмыгъэ')

[('тхузэмыгъэзахуэу', (267808,)), ('тхузэмыгъэкIуу', (164769,))]

100%|██████████| 430057/430057 [00:00<00:00, 3553509.39it/s]


In [2]:
import ahocorasick

list(store.data.word2word_id_dawg.items('тхузэфIэкIынут', '?', ahocorasick.MATCH_AT_MOST_PREFIX))

[('т', 6469),
 ('тх', 194093),
 ('тху', 5986),
 ('тхузэфIэкI', 27992),
 ('тхузэфIэкIын', 121246),
 ('тхузэфIэкIыну', 22258),
 ('тхузэфIэкIынут', 117580)]

In [7]:
list(store.data.word2word_id_dawg.items('тхузэфIэкIынут?', '?'))

[]

In [19]:
list(store.data.word2word_id_dawg.items('***эфIэкIынут', '*', ahocorasick.MATCH_AT_LEAST_PREFIX))

[('хузэфIэкIынут', 36107),
 ('хузэфIэкIынутэкъым', 36478),
 ('хузэфIэкIынутэкъыми', 143430),
 ('щызэфIэкIынутэкъым', 429094),
 ('ХузэфIэкIынутэкъым', 379224)]

In [3]:
list(store.find_by_suffix('кIынут', 3, False))

[('фIэкIынутэкъым', 289697),
 ('удукIынут', 187047),
 ('уаукIынут', 265027),
 ('щIэкIынут', 405201),
 ('щIэкIынутэкъым', 100759),
 ('сиукIынут', 167030),
 ('саукIынут', 264834),
 ('стекIынутэкъым', 363987),
 ('ЛъэкIынутэкъым', 352766),
 ('лъэкIынут', 363389),
 ('блэкIынут', 109299),
 ('блэкIынутэкъым', 154417),
 ('къикIынут', 219417),
 ('къикIынутэкъым', 148866),
 ('къэкIынутэкъым', 65538)]

In [9]:
len(store.data.tokenized_index[283])

13878

In [14]:
encode = store.data.tokenizer.encode('тхузэфIэкIынут')
result = {}

for step, (token_id, token) in enumerate(sorted(zip(encode.ids, encode.tokens), key=lambda x: x[0])):
    if step == 0:
        result = store.data.tokenized_index[token_id]
    else:
        result = result.intersection(store.data.tokenized_index[token_id])
    print(len(result))

80687
8131
8131
562
74
23
16
15
14


In [3]:
result

{22258,
 36107,
 52771,
 63459,
 113820,
 115912,
 117580,
 149888,
 173035,
 244790,
 334052,
 338394,
 388323,
 428007}

In [3]:
store.search_in_tokenized_index('тхузэфIэкIынут', 60)

[('тхузэфIэкIынут', 100),
 ('хузэфIэкIынут', 96),
 ('тхузэфIэкIыну', 96),
 ('тхузэфIэкIынум', 93),
 ('тхузэфIэкIынур', 93),
 ('схузэфIэкIынут', 93),
 ('тхузэфIэкIынури', 90),
 ('тхузэфIэмыкIыну', 90),
 ('тхузэфIэмыкIынур', 87),
 ('тхузэфIэкIынуращи', 84),
 ('тхузэфIэкIынукъым', 84),
 ('ЩIытхузэфIэкIынур', 84),
 ('зэрытхузэфIэкIынум', 81),
 ('зэрытхузэфIэкIынур', 81),
 ('тхузэфIэкIынукъыми', 81),
 ('хузэхъуэкIынут', 79),
 ('тхузэфIэгъэкIынухэм', 79),
 ('зэрытхузэхэгъэкIынур', 65),
 ('тхузэмыгъэкIуу', 64),
 ('хузэщIэту', 61)]

In [7]:
from math import log

log(5000 / 4000, 2)

0.32192809488736235

In [23]:
def tokenize(word):
    return store.data.tokenizer.encode(word).ids


def calculate_token_weight(token_id):
    return token_id / store.data.tokenizer.get_vocab_size()


def weighted_fuzz_ratio(word1, word2):
    tokens1 = tokenize(word1)
    tokens2 = tokenize(word2)

    score = 0
    total_weight = sum(calculate_token_weight(t) for t in tokens1) + sum(calculate_token_weight(t) for t in tokens2)

    for token1 in tokens1:
        weight1 = calculate_token_weight(token1)
        if token1 in tokens2:
            weight2 = calculate_token_weight(token1)  # Используйте вес token1, так как он совпадает
            score += (weight1 + weight2)

    # Нормализация результата
    if total_weight == 0:
        return 0

    penalty = (sum(calculate_token_weight(t) for t in tokens1 if t not in tokens2) +
               sum(calculate_token_weight(t) for t in tokens2 if t not in tokens1)) / total_weight
    length_difference_penalty = abs(len(tokens1) - len(tokens2)) / max(len(tokens1), len(tokens2))
    score = score / total_weight - penalty - length_difference_penalty
    return score




In [24]:
for r, s in store.search_in_tokenized_index('тхузэфIэкIынут', 60):
    print(r, s, weighted_fuzz_ratio('тхузэфIэкIынут', r))

тхузэфIэкIынут 100 1.0
хузэфIэкIынут 96 0.895939417678548
тхузэфIэкIыну 96 0.895939417678548
тхузэфIэкIынум 93 0.9953271028037383
тхузэфIэкIынур 93 0.9976608187134502
схузэфIэкIынут 93 0.9734257654534951
тхузэфIэкIынури 90 0.9872018615474113
тхузэфIэмыкIыну 90 -0.2713651268003154
тхузэфIэмыкIынур 87 -0.162756052141527
тхузэфIэкIынуращи 84 0.5647173489278753
тхузэфIэкIынукъым 84 0.9094466182224705
ЩIытхузэфIэкIынур 84 0.11091811414392053
зэрытхузэфIэкIынум 81 0.6419587776768205
зэрытхузэфIэкIынур 81 0.6438795538321602
тхузэфIэкIынукъыми 81 0.7956714761376249
хузэхъуэкIынут 79 0.38757655293088367
тхузэфIэгъэкIынухэм 79 0.7107025938050868
зэрытхузэхэгъэкIынур 65 -0.043229497774952275
тхузэмыгъэкIуу 64 -0.8138620936048587
хузэщIэту 61 -0.47525829255029906


In [10]:


for r, s in store.search_in_tokenized_index('зэфIэкIынут', 60):
    print(r)
    for tmpl, stem in zip(store.get_templates(r), store.stem_rule.find_possible_stems(r)):
        print('\t', tmpl, stem)
    print(list(store.find_sub_words(r)))
    set(store.stem_rule.find_possible_stems(r))
    print(
        set(store.stem_rule.find_possible_stems(r)).intersection(
            set([sw for sw, sw_id in store.find_sub_words(r)])
        )
    )

зэфIэкIынут
	 зэфIэкI* ынут
	 зэфIэ* кIынут
	 *ынут зэфIэкI
	 зэфI* экIынут
	 зэ*т фIэкIыну
	 *нут зэфIэкIы
	 *** зэфIэкIын
	 зэ* фIэкIынут
	 з*т эфIэкIыну
	 *ут зэфIэкIын
[('з', 7693), ('зэ', 120), ('зэфI', 292675), ('зэфIэкI', 1363), ('зэфIэкIын', 23081), ('зэфIэкIыну', 19321), ('зэфIэкIынут', 69409)]
{'зэфIэкI', 'зэфIэкIын'}
хузэфIэкIынут
	 хузэфIэ* кIынут
	 *ынут хузэфIэкI
	 хузэ* фIэкIынут
	 *нут хузэфIэкIы
	 ху*т зэфIэкIыну
	 *** хузэфIэкIын
	 ху* зэфIэкIынут
	 *ут хузэфIэкIын
[('х', 7522), ('ху', 4190), ('хузэфIэкI', 6566), ('хузэфIэкIын', 19127), ('хузэфIэкIыну', 18830), ('хузэфIэкIынут', 36107)]
{'хузэфIэкIын', 'хузэфIэкI'}
тхузэфIэкIынут
	 тхузэ* фIэкIынут
	 *ынут тхузэфIэкI
	 *нут тхузэфIэкIы
	 тху* зэфIэкIынут
	 *** тхузэфIэкIын
	 *ут тхузэфIэкIын
	 т*т хузэфIэкIыну
[('т', 6469), ('тх', 194093), ('тху', 5986), ('тхузэфIэкI', 27992), ('тхузэфIэкIын', 121246), ('тхузэфIэкIыну', 22258), ('тхузэфIэкIынут', 117580)]
{'тхузэфIэкIын', 'тхузэфIэкI'}
схузэфIэкIынут
	 схузэ* фIэкIыну

In [12]:
list(store.find_sub_words('IэгухэмкIэ'))

[('т', 6469),
 ('тх', 194093),
 ('тху', 5986),
 ('тхузэфIэкI', 27992),
 ('тхузэфIэкIын', 121246),
 ('тхузэфIэкIыну', 22258),
 ('тхузэфIэкIынур', 115912),
 ('тхузэфIэкIынуращи', 338394)]

In [1]:
from dags.src.stemmer.store import StemmerStore

store = StemmerStore()
store.load_data_from_file('../dags/src/stemmer/data/store_data.pkl')

100%|██████████| 430057/430057 [00:00<00:00, 1465910.60it/s]


In [10]:
from pprint import pprint

word = 'игъэкIуэдыртэкъым'
pprint(store.find_possible_stems(word, exist_only=False))
pprint(store.find_possible_stems(word, exist_only=True))
pprint(list(store.search_in_tokenized_index(word)))
pprint(list(store.find_sub_words(word)))

['игъэкIуэд',
 'кIуэдыртэ',
 'игъэкIуэды',
 'гъэкIуэдыр',
 'игъэкIуэдыр',
 'игъэкIуэдырт',
 'гъэкIуэдыртэ',
 'игъэкIуэдыртэ',
 'кIуэдыртэкъым',
 'гъэкIуэдыртэкъ',
 'игъэкIуэдыртэкъ',
 'игъэкIуэдыртэкъ',
 'игъэкIуэдыртэкъ',
 'гъэкIуэдыртэкъы']
[]
[('игъэкIуэдыртэкъым', 100),
 ('игъэкIуэдыркъым', 94),
 ('ирагъэкIуэкIытэркъым', 81),
 ('ирагъэщIэфыртэкъым', 80)]
[('и', 1), ('иг', 116747), ('игъэкI', 374779), ('игъэкIуэдыртэкъым', 105313)]


In [11]:
store.calc_stem(word, sw_min_len=4)

[('игъэкIуэдыркъым', 8)]

In [32]:
sorted(store.data.prefixes.items(), key=lambda x: x[1])

[('къыщыI', 126),
 ('къытте', 126),
 ('кърихь', 126),
 ('къэсщ', 127),
 ('къуле', 127),
 ('къис', 127),
 ('зэпры', 127),
 ('зыхуигъэ', 127),
 ('зыкъыс', 127),
 ('щызэхуэ', 127),
 ('щыхъу', 127),
 ('шыг', 127),
 ('къыхуигъ', 128),
 ('къыск', 128),
 ('къызэрыте', 128),
 ('къуд', 128),
 ('къакIэ', 128),
 ('къоп', 128),
 ('зэриу', 128),
 ('зыщиг', 128),
 ('зыщигъ', 128),
 ('дыкъызэр', 128),
 ('игъэг', 128),
 ('пфIэ', 128),
 ('ехъу', 128),
 ('жаI', 128),
 ('апхуэ', 128),
 ('Ег', 128),
 ('Шу', 128),
 ('Ре', 128),
 ('щхьэгъ', 128),
 ('узыщ', 128),
 ('уим', 128),
 ('уимы', 128),
 ('упэ', 128),
 ('къызэрык', 129),
 ('къызэфIэ', 129),
 ('къылъы', 129),
 ('зэхэп', 129),
 ('зэкъу', 129),
 ('зыIэр', 129),
 ('ехъ', 129),
 ('Сызэры', 129),
 ('макъ', 129),
 ('нэкIу', 129),
 ('Хъы', 129),
 ('апх', 129),
 ('апху', 129),
 ('Че', 129),
 ('симы', 129),
 ('щыщы', 129),
 ('щIэгъэ', 129),
 ('щIыщ', 129),
 ('узи', 129),
 ('къыхуэз', 130),
 ('къытез', 130),
 ('къыпте', 130),
 ('къыбгъэдэ', 130),
 ('къищI', 130)